In [1]:
import pandas as pd

In [2]:
df = pd.read_csv("../data/output/analytical.csv", low_memory=False)
df.head()

,casenumber,incident_date,death_date,age,gender,race,latino,cold_related,heat_related,commissioner_district,...,toxic_primary,toxic_secondary,tramadol_primary,tramadol_secondary,u-47700_primary,u-49900_primary,valeryl_primary,xylazine_primary,xylazine_secondary,nitazene
0,ME2022-06565,2022-07-12T02:38:00.000,2022-07-12,74.0,Male,White,0,0,0,4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
1,ME2022-06564,2022-07-11T23:46:00.000,2022-07-12,NaN,Male,Black,0,0,0,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
2,ME2022-06563,2022-07-02T00:00:00.000,2022-07-11,28.0,Male,Black,0,0,0,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
3,ME2022-06562,2022-07-11T20:48:00.000,2022-07-11,62.0,Female,White,0,0,0,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
4,ME2022-06561,2022-07-11T19:50:00.000,2022-07-11,62.0,Male,White,0,0,0,13.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0


In [3]:
df['death_date'] = pd.to_datetime(df['death_date'])
total_records = df.shape[0]
df = df[df['death_date'] < pd.to_datetime('07-01-2022')]
print(f"{total_records - df.shape[0]} records removed")

284 records removed


In [4]:
control1 = df[(df['fentanyl_primary'] == 1) | (df['fentanyl_secondary'] == 1)].copy()
case = control1[(control1['xylazine_primary'] == 1) | (control1['xylazine_secondary'] == 1)].copy()
control2 = df[
    ((df['eth_alc_primary'] == 1) | (df['eth_alc_secondary'] == 1))
    & ((df['opiate_primary'] != 1) | (df['opiate_secondary'] != 1))
].copy()

In [5]:
case['case_control'] = "xylazine"
control1['case_control'] = "fentanyl"
control2['case_control'] = "alcohol"

columns = ['casenumber', 'age', 'gender', 'race', 'final_latitude', 'final_longitude', 'case_control']

combined = pd.concat(
    [
        data[columns].copy() for data in [case, control1, control2]
    ], 
    ignore_index=True
).dropna()
print(combined.shape)
combined.head()


(10706, 7)


,casenumber,age,gender,race,final_latitude,final_longitude,case_control
0,ME2022-05956,38.0,Female,White,41.865180,-87.745063,xylazine
1,ME2022-05889,58.0,Male,Black,41.750815,-87.616426,xylazine
2,ME2022-05585,29.0,Female,Asian,41.973232,-87.753848,xylazine
3,ME2022-05537,63.0,Male,Black,41.983437,-87.837537,xylazine
4,ME2022-05477,66.0,Male,Black,41.733294,-87.837698,xylazine


In [6]:
combined.to_csv("combined.csv", index=False)